In [ ]:
import numpy as np
import extracted_data_reader
from tsai.all import *

# Training the model

In [ ]:
# initialization variables
show_splitting_dist = False
show_images = True
find_lr = False

ex_number = 1
frame_number = "max" # mean, max or min
epochs_number = 20
batch_size = [32, 64, 128]

In [ ]:
# get training data
X = np.array(extracted_data_reader.read_data_ex(ex_number=ex_number, frame_number=frame_number))
Y = np.array(extracted_data_reader.read_target_ex(ex_number=ex_number))
y = np.array([y[:5] for y in Y])

X.shape, y.shape

In [ ]:
# split data with leave-one-out cross-validation 
splits = get_splits(y, n_splits=len(y), shuffle=False, show_plot=show_splitting_dist)

In [ ]:
# create the datasets
tfms = [None, TSClassification()]
datasets = []
for split in splits:
    datasets.append(TSDatasets(X, y, splits=split, tfms=tfms, inplace=True))

In [ ]:
# create the data loader from the datasets
dls = TSDataLoaders.from_dsets(*datasets, bs=batch_size, batch_tfms=[TSStandardize()], num_workers=0)
if show_images: dls.show_batch(sharey=True)

In [ ]:
# create learner of an InceptionTimePlus model
learn = ts_learner(dls, 'InceptionTimePlus', metrics=accuracy)

In [ ]:
# fit the model
learn.fit_one_cycle(epochs_number, 1e-3)

In [ ]:
# save metrics images of the model 
image_base_name = f"C:/Users/Gabriele/Downloads/Uni/IOR-Rehab-Thesis/images/newmodels/ex{ex_number}/ex{ex_number}_{frame_number}"
learn.recorder.plot_metrics(figname=f"{image_base_name}_metrics.png")
learn.show_results(figname=f"{image_base_name}_results.png")
# learn.show_probas(figname=f"{image_base_name}_probas.png")
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figname=f"{image_base_name}_confusion_matrix.png")

In [ ]:
# save the model
learn.export(f"models/ex{ex_number}_{frame_number}.pkl")

# Testing the model

In [ ]:
# initialize testing variables
ex_number_test = 1
frame_number_test = "mean"

In [ ]:
# get testing data
X_test = np.array(extracted_data_reader.read_data_ex(ex_number_test, frame_number_test, test=True))

In [ ]:
# load the model 
learn = load_learner(f"models/ex{ex_number_test}_{frame_number_test}.pkl")

In [ ]:
# predict results
probas, target, preds = learn.get_X_preds(X_test)
preds